# Extract-Transform-Load Script

Extract-Transform-Load Scripts (ETLS) are common tools in data management. The purpose of ETLS is to gather relevant data (both direct and inferred) from public databases and capture important features in a possibly different data structure schema for specific analysis.

## PubMed Central ETLS Example

This script will Extract data from the CSV files provided to us by Stanford, Transform the data into a format usable by GeneDive, and then Load the data into the GeneDive sqlite database.

Whenever new data is obtained for GeneDive, this process should be run against that dataset. 

In [1]:
import re
import sqlite3
from shutil import copy2

In [2]:
# Progress Bar I found on the internet.
# https://github.com/alexanderkuk/log-progress
from progress_bar import log_progress

## <span style="color:red">IMPORTANT!</span> You need to create folders and organize the data before starting

Below are many file names and directory names. You have to create the directories and put the files correctly in them.

`GENE_GENE_INTERACTIONS_FILE`, `GENE_DRUG_INTERACTIONS_FILE`, and `GENE_DISEASE_INTERACTIONS_FILE` are TSV files from Emily. If they come with a .csv extension, and they are tab seperated, rename them. If they are deliminated some other way, change their extensions appropriately and change the value of `DELIMITER` below.

`GOOD_PHARM_GKB_DB`, `GOOD_ALL_DB`, and `GOOD_PLOS_PMC_DB` are the current working, valid databases used in GeneDive. They will not be altered, but instead they will be copied and updated.

`PLOS_PMC_DB` and `ALL_DB` are the newly generated databases.

In [3]:
# TSV files containing Emily's data
GENE_GENE_INTERACTIONS_FILE = 'tsv_data/genegene_relationship_db_sfsu_with_excerpts.tsv'
GENE_DRUG_INTERACTIONS_FILE = 'tsv_data/genedrug_relationship_100417_sfsu_with_excerpts.tsv'
GENE_DISEASE_INTERACTIONS_FILE = 'tsv_data/genedisease_relationship_100417_sfsu_with_excerpts.tsv'

# These will be unaltered
GOOD_PHARM_GKB_DB = 'sqlite_data/good_data/data.pgkb.sqlite'
GOOD_ALL_DB = 'sqlite_data/good_data/data.all.sqlite'
GOOD_PLOS_PMC_DB = 'sqlite_data/good_data/data.plos-pmc.sqlite'

# These will be created
PLOS_PMC_DB = 'sqlite_data/data.plos-pmc.sqlite' # This will be just the data from emilies files
ALL_DB = 'sqlite_data/data.all.sqlite' # This is a combination of emilies files and PharmGKB

DELIMITER = "\t"
EMILYS_FILES = [
#    GENE_GENE_INTERACTIONS_FILE,
    GENE_DRUG_INTERACTIONS_FILE, 
    GENE_DISEASE_INTERACTIONS_FILE,
]

If write is false, the script will run but not write anything to the database. This keeps it safe while you're nosing around, and can also be useful if you need to re-generate the complete typeahead/adjacency files.

In [4]:
WRITE = True

This copies the fields, then initializes the connections. The `databases` array will be looped through at the end, applying all the interactions to each database.

In [5]:
copy2(GOOD_PLOS_PMC_DB, PLOS_PMC_DB)
conn_plos_pmc = sqlite3.connect(PLOS_PMC_DB)
cursor_plos_pmc = conn_plos_pmc.cursor()

copy2(GOOD_ALL_DB, ALL_DB)
conn_all = sqlite3.connect(ALL_DB)
cursor_all = conn_all.cursor()


databases = [
    { "name": "PLOS-PMC","conn": conn_plos_pmc, "cursor": cursor_plos_pmc,}, 
    { "name": "ALL","conn": conn_all, "cursor": cursor_all}
]

Map the columns as they appear in the file to the correct values.

In [6]:
# If the exercept is not found, don't run the Excerpt wrapping cell below
excerptFound = False
interactions = []
for filename in EMILYS_FILES:
    with open(filename) as file:
        header = None
        linenum = 0  
        for line in file:
            linenum+=1
            
            # Read the headers of the file and assign them to a dictionary {column_name: column_number}
            if linenum == 1:
                header = {name.strip(): col for col, name in enumerate(line.split(DELIMITER))}
                
                # The GeneGene headers differ from Gene Drug and Gene Disease. This normalizes them.
                if "geneids" in header and "disease_ids" in header: # GeneDrug/GeneDisease
                    header["dgd1"] = header["geneids"]
                    header["dgd2"] = header["disease_ids"]
                    header["mention1_offset"] = header["mention1_offset_start"]
                    header["mention2_offset"] = header["mention2_offset_start"]
                elif "geneids1" in header and "geneids2" in header: # GeneGene
                    header["dgd1"] = header["geneids1"]
                    header["dgd2"] = header["geneids2"]
                else:
                    raise ValueError('{f} column headers didn\'t contain expected values'.format(f = filename))
                
                # if no excerpts provided, substituted with article name        
                if "excerpt" in header:
                    
                    excerptFound = True
                else:
                    header["excerpt"] = header["article_id"]
                
                #print(header)
                    
                continue
                
            line = line.strip().split(DELIMITER)

            interaction = {
              "journal": line[header["journal"]], # no change
              "article_id": line[header["article_id"]], # no change
              "pubmed_id": line[header["pubmed_id"]], # no change
              "sentence_id": line[header["sentence_id"]], # no change
              "mention1_offset": line[header["mention1_offset"]], # new data describes a mention1_offset_start and mention1_offset_end -- I arbitarily chose to just assign offset_start here (offset start and end are often the same anyway) 
              "mention2_offset": line[header["mention2_offset"]], # same principle as above, but for mention2
              "mention1": line[header["mention1"]], # no change
              "mention2": line[header["mention2"]], # no change
              "geneids1": line[header["dgd1"]], # there's a column named "geneids", but it never seems to contain more than one value "MESH:xxxxxxx"
              "geneids2": line[header["dgd2"]], # the column after "geneids" is called "disease_ids", and may be a suitable substitute for this geneids value
              "probability": line[header["probability"]], # no change
              "excerpt": line[header["excerpt"]] 
            }

            interactions.append(interaction)
totalInteractions = len(interactions)


In [7]:
print(interactions[100])

{'sentence_id': 'SENT121', 'mention1_offset': '25', 'mention2': 'p38', 'pubmed_id': '25541625', 'excerpt': 'ERKs, JNKs, and #p38# have consistently been identified in platelets [20] and they are activated in platelets stimulated by collagen and #thrombin# and are involved in thrombosis [21].', 'article_id': 'ScientificWorldJournal_2014_Oct_14_2014_745802.nxml.txt.nlp', 'probability': '1', 'geneids2': '10598;1432;7965', 'mention2_offset': '5', 'geneids1': 'PA451674', 'journal': 'pmc', 'mention1': 'thrombin'}


Remove any interactions for which the a gene traces to multiple IDs.

In [8]:

interactions = [i for i in interactions if ( ';' not in i['geneids1'] and ';' not in i['geneids2'])]
interactions = [i for i in interactions if ('NULL' not in i['article_id'] and 'NULL' not in i['pubmed_id'] and 'NULL' not in i['sentence_id'] and 'NULL' not in i['mention1_offset'] and 'NULL' not in i['mention2_offset'] and 'NULL' not in i['mention1'] and 'NULL' not in i['mention2'] and 'NULL' not in i['geneids1'] and 'NULL' not in i['geneids2'] and 'NULL' not in i['probability'] and 'NULL' not in i['excerpt'])] #there's probably a shorter way of doing this, but I think this works for now -- excludes JOURNAL entries of course


newTotal = len(interactions)
print(
'''Total Interactions:     {total}
Filtered Interactions:  {filtered}
Remaining Interactions: {remaining}'''
      .format(total = totalInteractions,filtered=totalInteractions-newTotal , remaining =newTotal))


Total Interactions:     584939
Filtered Interactions:  60903
Remaining Interactions: 524036


## Excerpt wrapping
GeneDive expects the target genes in the excerpt to be wrapped in pound signs. This is important because a sentence may mention the target gene multiple times, so we need to use the offset data her to make sure we tag the right mention.

In [9]:
# disabled because the data already came tokenized. Remove "and False" to re-enable
if excerptFound and False:
    for i in interactions:
        try:
            if (i['journal'] != 'journal'):
                excerpt = i['excerpt']

                excerpt = re.sub('"', '', excerpt)
                tokens = excerpt.split(" ")
                offset1 = int(i['mention1_offset'])
                offset2 = int(i['mention2_offset'])

                tokens[offset1] = "".join(["#",tokens[offset1],"#"])
                tokens[offset2] = "".join(["#",tokens[offset2],"#"])

                i['excerpt'] = " ".join(tokens)
        except Exception as error:
            print(i)
            raise error

**Specific for PMC Data**

We didn't get Journal Data - we need to extract it from the article titles. Comment out the next section if journal titles were included.

In [12]:
for i in interactions:
    toks = i['article_id'].split("_")
    toks_len_minus_one = len(toks) - 1

    for j in range(toks_len_minus_one+1):
        if toks[j][:2] == "19" or toks[j][:2] == "20":
            journal = " ".join(toks[:j])
            break
        if(j == toks_len_minus_one): # If last entry, just make it N/A
            journal = "N/A"

    i['journal'] = journal

Our insert statement - probably don't need to touch this

In [13]:
INTERACTIONS_WRITE = '''insert into interactions ( journal, article_id, pubmed_id, sentence_id, mention1_offset, mention2_offset, mention1, mention2, geneids1, geneids2, probability, context, section, reactome ) values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? );'''

## Add the data to sqlite files

This will load the data into the plos-pmc database and the all database.

In [14]:
for sql in databases:
    for interaction in log_progress(interactions, every=100, name="{name} insertions".format(name=sql["name"])):
        
        values = (
                    interaction['journal'],
                    interaction['article_id'],
                    interaction['pubmed_id'],
                    interaction['sentence_id'],
                    interaction['mention1_offset'],
                    interaction['mention2_offset'],
                    interaction['mention1'],
                    interaction['mention2'],
                    interaction['geneids1'],
                    interaction['geneids2'],
                    interaction['probability'],
                    interaction['excerpt'],
                    "Unknown",
                    0,
                )
        
        try:
            sql["cursor"].execute(INTERACTIONS_WRITE, values )
        except Exception as error:
            print("interactions:\t",interaction)
            print("values:\t",values)
            raise error

    if WRITE:
        sql["conn"].commit()
        print("Database write complete")
        
    sql["conn"].close()
    print("Connection closed")




VBox(children=(HTML(value=''), IntProgress(value=0, max=524036)))

VBox(children=(HTML(value=''), IntProgress(value=0, max=524036)))